* 一到多個key(可以是函式、陣列或DataFrame欄位名稱)將一個pandas物件切成多個
* 計算分組彙總統計值,像是個數、平均值、標準差或使用者自定值
* 對組進行轉換或其他操作,例如:正規化、線性迴歸或是選擇子集
* 計算樞紐分析表或是交叉表
* 進行分位數分析或是其他統計分析

# GroupBy 機制
* split apply combine
  * split 分裂:`分割by key`
  * apply 套用:`統計量sum mean...`
  * cobine 合併:`類似r 套件summarise`
* 分組的key的特徵
  * 可以是含值的list或array,只要長度和要用來分組的軸相等
  * DataFrame中一個欄的名稱
  * 可以是dict或Series,指定分組是該用哪些值,以及對應的組名稱
  * 可以是函式,依每個軸索引或每個索引標籤呼叫


In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
          'key2':['one','two','one','two','one'],
          'data1':np.random.randn(5),
          'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.530238,-0.655893
1,a,two,1.936801,1.189714
2,b,one,-0.426496,-1.546852
3,b,two,0.673688,1.669886
4,a,one,1.286376,0.184663


In [18]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [19]:
grouped.mean()

key1
a    0.897646
b    0.123596
Name: data1, dtype: float64

In [20]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one     0.378069
      two     1.936801
b     one    -0.426496
      two     0.673688
Name: data1, dtype: float64

In [21]:
means.unstack()

key2,one,two
key1,,
a,0.378069,1.936801
b,-0.426496,0.673688


In [22]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005    1.936801
            2006   -0.426496
Ohio        2005    0.071725
            2006    1.286376
Name: data1, dtype: float64

In [23]:
df.groupby([states,years]).mean()

data1     data2
California 2005  1.936801  1.189714
           2006 -0.426496 -1.546852
Ohio       2005  0.071725  0.506996
           2006  1.286376  0.184663

In [24]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.897646,0.239495
b,0.123596,0.061517


In [26]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.378069 -0.235615
     two   1.936801  1.189714
b    one  -0.426496 -1.546852
     two   0.673688  1.669886

In [27]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 疊代分組

In [29]:
for name,group in df.groupby('key1'):#按照順序分組顯示
  print(name)
  print(group)

a
  key1 key2     data1     data2
0    a  one -0.530238 -0.655893
1    a  two  1.936801  1.189714
4    a  one  1.286376  0.184663
b
  key1 key2     data1     data2
2    b  one -0.426496 -1.546852
3    b  two  0.673688  1.669886


In [32]:
for (k1,k2),group in df.groupby(['key1','key2']):#整合單位細格
  print((k1,k2))
  print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.530238 -0.655893
4    a  one  1.286376  0.184663
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.936801  1.189714
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.426496 -1.546852
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.673688  1.669886


In [34]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.426496,-1.546852
3,b,two,0.673688,1.669886


In [35]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [37]:
grouped = df.groupby(df.dtypes,axis=1)
grouped

In [38]:
for dtype,group in grouped:
  print(dtype)
  print(group)

float64
      data1     data2
0 -0.530238 -0.655893
1  1.936801  1.189714
2 -0.426496 -1.546852
3  0.673688  1.669886
4  1.286376  0.184663
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 選取一或多個欄

In [39]:
df.groupby('key1')['data1']

In [40]:
df['data1'].groupby(df['key1'])

In [41]:
df.groupby('key1')[['data2']]

In [44]:
df[['data2']].groupby(df['key1'])##都是等價

In [46]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.235615
     two   1.189714
b    one  -1.546852
     two   1.669886

In [47]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [48]:
s_grouped.mean()

key1  key2
a     one    -0.235615
      two     1.189714
b     one    -1.546852
      two     1.669886
Name: data2, dtype: float64

## 用Dict和Series進行分組

In [50]:
people = pd.DataFrame(np.random.randn(5,5),
            columns=['a','b','c','d','e'],
            index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan# 加入幾個NA值
people

,a,b,c,d,e
Joe,-0.799310,0.868913,0.320056,0.116742,0.023032
Steve,0.096596,0.077413,-0.558728,-0.978733,-1.636078
Wes,0.643173,NaN,NaN,0.040677,0.198563
Jim,-1.432659,-0.378374,0.395543,0.333982,0.272461
Travis,-1.764857,-0.197008,0.225247,-0.303584,-1.145828


In [51]:
mapping = {'a':'red','b':'red','c':'blue',
       'd':'blue','e':'red','f':'orange'}

In [53]:
-0.799310+0.868913+0.320056+0.116742+0.023032

0.5294330000000002

In [56]:
-0.799310+0.868913+0.023032#red

0.09263500000000008

In [52]:
by_column = people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,0.436798,0.092635
Steve,-1.537461,-1.462069
Wes,0.040677,0.841736
Jim,0.729526,-1.538572
Travis,-0.078337,-3.107694


In [61]:
map_series = pd.Series(mapping)#當作分組固定對應看待
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [63]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 用函式分組

In [65]:
people.groupby(len).sum()#用人名長度分組

,a,b,c,d,e
3,-1.588796,0.490539,0.715599,0.491402,0.494057
5,0.096596,0.077413,-0.558728,-0.978733,-1.636078
6,-1.764857,-0.197008,0.225247,-0.303584,-1.145828


In [67]:
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.799310  0.868913  0.320056  0.040677  0.023032
  two -1.432659 -0.378374  0.395543  0.333982  0.272461
5 one  0.096596  0.077413 -0.558728 -0.978733 -1.636078
6 two -1.764857 -0.197008  0.225247 -0.303584 -1.145828

## 用索引層級分組

In [69]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                    [1,3,5,1,3]],
                    names=['cty','tenor'])
hief_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
hief_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.782257 -0.378239 -0.518441 -0.973883 -0.125867
1      0.270589 -0.217892  0.827698  0.565833 -1.047163
2     -0.742736 -0.770834  0.221705 -0.630574  1.636739
3     -0.083996  0.532859 -0.111629 -1.609645 -0.116346

In [70]:
hief_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 資料聚合


* 做過優化的groupby方法

| 🔥函式名稱 | 🐤描述 |
| -------- |:------:|
|count|計算組中non-NA的值個數|
|sum|將non-NA的值加總|
|mean|將non-NA的值做平均|
|median|將non-NA的值做算術中位數|
|std,var|無偏(分母為n-1)標準差和變異數|
|min,max|non-NA值中最小值和最大值|
|prod|取nan-NA值乘積|
|first,last|第一個和最後一個nan-NA值|

In [71]:
df

,key1,key2,data1,data2
0,a,one,-0.530238,-0.655893
1,a,two,1.936801,1.189714
2,b,one,-0.426496,-1.546852
3,b,two,0.673688,1.669886
4,a,one,1.286376,0.184663


In [73]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.806716
b    0.563670
Name: data1, dtype: float64

In [75]:
def peak_to_peak(arr):
  return arr.max() -arr.min()
grouped.agg(peak_to_peak)

/usr/local/lib/python3.8/dist-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,2.467039,1.845608
b,1.100184,3.216738


In [77]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.897646  1.278633 -0.530238  0.378069  1.286376  1.611589   
b      2.0  0.123596  0.777948 -0.426496 -0.151450  0.123596  0.398642   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.936801   3.0  0.239495  0.924025 -0.655893 -0.235615  0.184663   
b     0.673688   2.0  0.061517  2.274577 -1.546852 -0.742668  0.061517   

                          
           75%       max  
key1                      
a     0.687189  1.189714  
b     0.865701  1.669886

## 欄方向的多功能應用

In [79]:
#修改工作路徑
import os
from google.colab import drive
drive.mount('/content/drive')

path="/content/drive/MyDrive/python/dataset"
os.chdir(path)
os.listdir()#判斷裡面內容

Mounted at /content/drive


['iris.csv',
 'starwars.xlsx',
 'airquality.csv',
 'mtcars.csv',
 'diamonds.csv',
 'out.csv',
 'banklist.csv',
 'Performance_MNR.xml',
 'ratings.csv',
 'movies.csv',
 'tags.csv',
 'links.csv',
 'macrodata.txt',
 'spx.csv',
 'tips.csv',
 'macrodata.csv']

In [80]:
tips = pd.read_csv('tips.csv')

In [82]:
# 加入小費占總結帳比例
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [83]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [85]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [86]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [87]:
functions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(functions)
result

<ipython-input-87-05231bb63e58>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct','total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [88]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [90]:
ftuples = [('Durchschnitt','mean'),('Abweichung',np.var)]
grouped['tip_pct','total_bill'].agg(ftuples)

<ipython-input-90-13db5c96125f>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct','total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [91]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [93]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 回傳聚合資料時不要列索引

In [95]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# Apply:分裂-套用-合併

In [97]:
def top(df,n=5,column='tip_pct'):
  return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [98]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [99]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [101]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [104]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [107]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

## 分位數與籃(bucket)分析

In [110]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
            'data2':np.random.randn(1000)})
quartiles = pd.cut(frame.data1,4)
quartiles[:10]

0     (-0.299, 1.371]
1     (-0.299, 1.371]
2     (-0.299, 1.371]
3     (-0.299, 1.371]
4     (-0.299, 1.371]
5       (1.371, 3.04]
6    (-1.969, -0.299]
7     (-0.299, 1.371]
8     (-0.299, 1.371]
9    (-1.969, -0.299]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.646, -1.969] < (-1.969, -0.299] < (-0.299, 1.371] <
                                           (1.371, 3.04]]

In [113]:
def get_stats(group):
  return {'min':group.min(),'max':group.max(),
       'count':group.count(),'mean':group.mean()}

In [114]:
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.646, -1.969]",-1.323898,2.161642,24.0,-0.071842
"(-1.969, -0.299]",-2.656900,2.499233,370.0,0.004571
"(-0.299, 1.371]",-3.290715,3.359731,517.0,-0.051475
"(1.371, 3.04]",-2.080589,2.552168,89.0,0.023225


## 範例:依分組指定填充遺失值

In [116]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.084019
2         NaN
3   -1.817058
4         NaN
5   -0.972545
dtype: float64

In [117]:
s.fillna(s.mean())

0   -0.901861
1    0.084019
2   -0.901861
3   -1.817058
4   -0.901861
5   -0.972545
dtype: float64

In [119]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key = ['East'] * 4+ ['West'] * 4
data = pd.Series(np.random.randn(8),index=states)
data

Ohio          0.536498
New York     -0.282341
Vermont      -0.744035
Florida       1.326792
Oregon       -0.733623
Nevada        0.370721
California    0.391375
Idaho        -1.085395
dtype: float64

In [121]:
data[['Vermont','Nevada','Idaho']]=np.nan
data

Ohio          0.536498
New York     -0.282341
Vermont            NaN
Florida       1.326792
Oregon       -0.733623
Nevada             NaN
California    0.391375
Idaho              NaN
dtype: float64

In [123]:
data.groupby(group_key).mean()

East    0.526983
West   -0.171124
dtype: float64

In [124]:
fill_mean = lambda g : g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.536498
New York     -0.282341
Vermont       0.526983
Florida       1.326792
Oregon       -0.733623
Nevada       -0.171124
California    0.391375
Idaho        -0.171124
dtype: float64

In [125]:
fill_values = {'East':0.5,'West':-1}

In [127]:
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.536498
New York     -0.282341
Vermont       0.500000
Florida       1.326792
Oregon       -0.733623
Nevada       -1.000000
California    0.391375
Idaho        -1.000000
dtype: float64

## 範例取樣和排列

In [131]:
# 紅心梅花黑桃和方塊
suits = ['H','S','C','D']
card_val = (list(range(1,11))+[10]*3) * 4
base_names = ['A']+list(range(2,11))+['J','K','Q']
cards = []
for suit in ['H','S','C','D']:
  cards.extend(str(num)+suit for num in base_names)
deck = pd.Series(card_val,index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [132]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [134]:
def draw(deck,n=5):
  return deck.sample(n)
draw(deck)#抽出五張

3H     3
8C     8
4D     4
6C     6
QD    10
dtype: int64

In [136]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw,n=2)#每個化色都抽2個

C  AC     1
   4C     4
D  5D     5
   KD    10
H  9H     9
   6H     6
S  AS     1
   6S     6
dtype: int64

In [137]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

AC     1
JC    10
8D     8
KD    10
7H     7
JH    10
5S     5
9S     9
dtype: int64

## 範例:加權平均和關聯性

In [138]:
df = pd.DataFrame({'category':['a','a','a','a',
                  'b','b','b','b'],
          'data':np.random.randn(8),
          'weights':np.random.randn(8)})
df

,category,data,weights
0,a,-1.626759,-1.148031
1,a,-0.173833,-0.096919
2,a,-0.270318,-1.433572
3,a,-0.070018,0.493388
4,b,0.819635,2.330580
5,b,0.112034,0.184346
6,b,-0.122123,0.115379
7,b,0.849162,-0.529138


In [143]:
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a   -1.023916
b    0.698404
dtype: float64

In [147]:
close_px = pd.read_csv('stock_px.csv',parse_dates= True,index_col=0)

In [148]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [149]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [150]:
spx_corr = lambda x : x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x :x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [153]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 範例:分組線性迴歸

In [159]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
  Y = data[yvar]
  X = data[xvars]
  X['intercept']=1.
  result = sm.OLS(Y,X).fit()
  return result.params

In [160]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 樞紐關係表和交叉表
* pivot_table

| 🔥功能名稱 | 🐤描述 |
| -------- |:------:|
|values|要聚合的名稱;預設是聚合所有數值欄位|
|index|欄位名稱或其他分組key,用來對列分組,以產生樞紐分析表|
|columns|欄位名稱或其他分組key,用來對欄分組,以產生樞紐分析表|
|aggfunc|用來聚合的函式,或函式們(預設是'mean');任何groupby可用的函式這裡都可以用|
|fill_value|在產出表格中用來代替遺失值的值|
|dropna|如果是True,捨棄整欄都是NA的欄|
|margins|加入列/欄的小計和加總(預設為False)|

In [161]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [164]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [166]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)#margin有合計的功能

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [167]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [168]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## 交叉表:Crosstab
```python
data
pd.crosstab(data.Nationality,data.Hanedness,margins=True)
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)
```

In [170]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244